In [1]:
import os
os.chdir('../../src/')
from visualization.SIR_Plot import plot_hospitalized

In [2]:
os.chdir('models')
from MCFS import mcts, treeNode
import simulate_pandemic as simp

Loading Graph...Done!


In [24]:
from glob import glob
import pickle as pkl
import pandas as pd

H = 3
N = 48
D = 364

files = glob(f'../../data/MCTS_Results/pickles/looser_cost_H{H}_N{N}_D{D}_bfFalse*.pkl')
results = []
print(len(files))
for file in files:
    with open(file, 'rb') as f:
        data, actions, tree = pkl.load(f)
    data = plot_hospitalized(data, actions, 7, file + '.pdf', make_df=False)
    #data = pd.DataFrame([pd.Series(d).value_counts() for d in data])
    #data.fillna(0, inplace=True)
    results.append(data)

1


In [25]:
from sklearn.metrics import mean_absolute_error
import numpy as np

pop=len(simp.G.nodes())
capacity = 0.0015


mae_policy = [mean_absolute_error(y_true=len(d)*[capacity], y_pred=d[3]/pop) for d in results]
print('MAE AVERAGE ', np.mean(mae_policy))
print('MAE STD ',np.std(mae_policy))

MAE AVERAGE  0.0006922997711579849
MAE STD  0.0


In [26]:
over_capacity = []
for r in results:
    tmp = (r[3] / pop) - capacity
    over_capacity.append(tmp[tmp>0].sum())

print('CAPACITY OVERLOAD MEAN ', np.mean(over_capacity))
print('CAPACITY OVERLOAD STD ', np.std(over_capacity))

CAPACITY OVERLOAD MEAN  0.0007585958336336766
CAPACITY OVERLOAD STD  0.0


In [27]:
from actions import costs, exposed_cost

replicated_actions = [l for lst in [7*[a] for a in actions] for l in lst]

sims_costs = []
for d in results:
    sims_costs.append(np.sum([max(exposed_cost(v/pop), costs[replicated_actions[i]]) for i, v in enumerate(d[3])]))

print(np.mean(sims_costs))
print(np.std(sims_costs))

57.12946931287378
0.0
